In [ ]:
%%capture
%pip install duckdb requests pandas

# Capítulo 06: Catálogos REST e Autenticação

Este notebook demonstra como trabalhar com catálogos REST Iceberg e autenticação OAuth2 no DuckDB.

## 📦 Instalação de Dependências

## 1️⃣ Catálogo REST com OAuth2

Configuração básica de um catálogo REST com autenticação OAuth2.

In [ ]:
import duckdb

con = duckdb.connect()
con.execute("INSTALL iceberg")
con.execute("LOAD iceberg")
con.execute("LOAD httpfs")

# Criar secret
con.execute("""
    CREATE SECRET my_iceberg_secret (
        TYPE iceberg,
        CLIENT_ID 'my_client_id',
        CLIENT_SECRET 'my_client_secret',
        OAUTH2_SERVER_URI 'https://catalog.example.com/v1/oauth/tokens'
    )
""")

# Anexar catálogo
con.execute("""
    ATTACH 'my_warehouse' AS iceberg_cat (
        TYPE iceberg,
        SECRET my_iceberg_secret,
        ENDPOINT 'https://catalog.example.com'
    )
""")

# Consultar tabelas
tables = con.execute("""
    SELECT * FROM iceberg_cat.information_schema.tables
""").df()

print(tables)

## 2️⃣ AWS Glue Catalog

Integração com AWS Glue como catálogo Iceberg.

In [ ]:
import duckdb

con = duckdb.connect()
con.execute("INSTALL iceberg")
con.execute("LOAD iceberg")
con.execute("LOAD httpfs")

# Configurar credenciais AWS
con.execute("""
    CREATE SECRET aws_secret (
        TYPE s3,
        PROVIDER credential_chain
    )
""")

# Anexar Glue Catalog
con.execute("""
    CREATE SECRET glue_secret (
        TYPE iceberg,
        CLIENT_ID 'aws_access_key',
        CLIENT_SECRET 'aws_secret_key',
        OAUTH2_SERVER_URI 'https://glue.us-east-1.amazonaws.com'
    )
""")

con.execute("""
    ATTACH 'glue_warehouse' AS glue_catalog (
        TYPE iceberg,
        SECRET glue_secret,
        ENDPOINT 'https://glue.us-east-1.amazonaws.com'
    )
""")

## 3️⃣ Listando Tabelas do Catálogo

Como explorar tabelas disponíveis no catálogo.

In [ ]:
import duckdb

con = duckdb.connect()
con.execute("INSTALL iceberg")
con.execute("LOAD iceberg")

# ... anexar catálogo ...

# Listar todas as tabelas
tables = con.execute("""
    SELECT table_catalog, table_schema, table_name
    FROM iceberg_cat.information_schema.tables
    WHERE table_type = 'BASE TABLE'
""").df()

print("Tabelas disponíveis:")
print(tables)

## 4️⃣ Consultando Dados via Catálogo

Executar queries nas tabelas do catálogo.

In [ ]:
import duckdb

con = duckdb.connect()

# ... configurar catálogo ...

# Consultar tabela via catálogo
result = con.execute("""
    SELECT *
    FROM iceberg_catalog.default.sales
    WHERE order_date >= '2024-01-01'
    LIMIT 1000
""").df()

print(result.head())

## 5️⃣ Credenciais via Variáveis de Ambiente

Gerenciar credenciais de forma segura usando variáveis de ambiente.

In [ ]:
import duckdb
import os

# Ler credenciais de variáveis de ambiente
client_id = os.getenv('ICEBERG_CLIENT_ID')
client_secret = os.getenv('ICEBERG_CLIENT_SECRET')
oauth_uri = os.getenv('ICEBERG_OAUTH_URI')

con = duckdb.connect()
con.execute("INSTALL iceberg")
con.execute("LOAD iceberg")

# Criar secret com variáveis de ambiente
con.execute(f"""
    CREATE SECRET env_secret (
        TYPE iceberg,
        CLIENT_ID '{client_id}',
        CLIENT_SECRET '{client_secret}',
        OAUTH2_SERVER_URI '{oauth_uri}'
    )
""")

## 6️⃣ Classe Gerenciadora de Catálogo

Implementação de uma classe para facilitar o gerenciamento de catálogos.

In [ ]:
import duckdb
import os

class IcebergCatalogManager:
    def __init__(self, endpoint, client_id, client_secret, oauth_uri):
        self.con = duckdb.connect()
        self.con.execute("INSTALL iceberg")
        self.con.execute("LOAD iceberg")
        self.con.execute("LOAD httpfs")

        # Criar secret
        self.con.execute(f"""
            CREATE SECRET catalog_secret (
                TYPE iceberg,
                CLIENT_ID '{client_id}',
                CLIENT_SECRET '{client_secret}',
                OAUTH2_SERVER_URI '{oauth_uri}'
            )
        """)

        # Anexar catálogo
        self.con.execute(f"""
            ATTACH 'warehouse' AS catalog (
                TYPE iceberg,
                SECRET catalog_secret,
                ENDPOINT '{endpoint}'
            )
        """)

    def list_tables(self, schema='default'):
        """Lista tabelas em um schema"""
        return self.con.execute(f"""
            SELECT table_name
            FROM catalog.information_schema.tables
            WHERE table_schema = '{schema}'
        """).df()

    def query_table(self, table_path, limit=100):
        """Consulta tabela do catálogo"""
        return self.con.execute(f"""
            SELECT *
            FROM {table_path}
            LIMIT {limit}
        """).df()

# Usar
catalog = IcebergCatalogManager(
    endpoint='https://catalog.example.com',
    client_id=os.getenv('CLIENT_ID'),
    client_secret=os.getenv('CLIENT_SECRET'),
    oauth_uri='https://catalog.example.com/v1/oauth/tokens'
)

# Listar tabelas
tables = catalog.list_tables('sales')
print("Tabelas:", tables)

# Consultar
data = catalog.query_table('catalog.sales.orders')
print(data.head())

## 7️⃣ Teste de Conectividade

Verificar conectividade com o catálogo REST.

In [ ]:
import duckdb
import requests

def test_catalog_connection(endpoint, token=None):
    """Testa conectividade com catálogo REST"""
    headers = {}
    if token:
        headers['Authorization'] = f'Bearer {token}'

    try:
        response = requests.get(f"{endpoint}/v1/config", headers=headers)
        print(f"✅ Catálogo acessível: {response.status_code}")
        return True
    except Exception as e:
        print(f"❌ Erro de conectividade: {e}")
        return False

# Testar
test_catalog_connection('https://catalog.example.com')

## 8️⃣ Teste Completo de Configuração

Validar todas as configurações de catálogo e autenticação.

In [ ]:
import duckdb

con = duckdb.connect()
con.execute("INSTALL iceberg")
con.execute("LOAD iceberg")

try:
    con.execute("""
        CREATE SECRET test_secret (
            TYPE iceberg,
            CLIENT_ID 'test_id',
            CLIENT_SECRET 'test_secret',
            OAUTH2_SERVER_URI 'https://catalog.example.com/oauth/tokens'
        )
    """)
    print("✅ Secret criado com sucesso")

    con.execute("""
        ATTACH 'test_warehouse' AS test_cat (
            TYPE iceberg,
            SECRET test_secret,
            ENDPOINT 'https://catalog.example.com'
        )
    """)
    print("✅ Catálogo anexado com sucesso")

except Exception as e:
    print(f"❌ Erro: {e}")